# Vodafone Input Debug

## Global import

In [ ]:
from global_import import *

## Platform initialization

In [ ]:
project_tourism = dh.get_or_create_project("overtourism1")
endpoint_url="http://minio:9000"

s3 = boto3.resource('s3',
                    endpoint_url=endpoint_url)

bucket = s3.Bucket('datalake')


## OD - parameters

In [ ]:
list_time_intervals = [[7,8],[8,9],[10,11],[16,17],[18,19],[25,26]]                                                                                                                                # NOTE: time intervals of interest in hours   
DEBUG = False
is_weekday = False
str_t = 7.0
str_t1 = 8.0
int_time_hour_start_window_interest = 7
user_profile = "AGGREGATED"
str_day = "2024-08-08"
col_str_day = col_str_day_od  
str_col_n_trips = f"n_trips_{user_profile}_t_{str_t}_{str_t1}"           

In [ ]:
list_files_od, list_files_presenze, list_str_dates_yyyymm = extract_filenames_and_date_from_bucket(bucket)
df_od_july = extract_od_vodafone_from_bucket(s3,list_files_od, 0)
df_od_august = extract_od_vodafone_from_bucket(s3,list_files_od, 1)
df_od_october = extract_od_vodafone_from_bucket(s3,list_files_od, 2)


In [ ]:
print("periods in july dataset: ",df_od_july["PERIOD_ID"].unique(),"\nperiods in august dataset: ",df_od_august["PERIOD_ID"].unique(),"\nperiods in october dataset: ",df_od_october["PERIOD_ID"].unique())
list_str_dates_yyyymm

In [ ]:
# Test the date extraction functions
print("Testing date extraction functions...")
print("=" * 50)

# Test samples from your data
test_samples = [
    # July dataset samples  
    "202408 01-15 - Feriale",
    "202407 16-31 - Festivo", 
    "202407 01-15 - Feriale",
    "202407 01-15 - Festivo",
    "202407 16-31 - Feriale",
    # August dataset samples
    "202408 01-15 - Festivo",
    "202408 16-31 - Festivo", 
    "202408 01-15 - Prefestivo",
    # October dataset samples (null day format)
    "202410 - Prefestivo",
    "202410 - Feriale", 
    "202410 - Festivo"
]

print("Testing extract_date_info (for July/August):")
for sample in test_samples[:7]:
    result = extract_date_info(sample)
    if result is not None:
        date_str, is_weekday = result
        print(f"  '{sample}' -> Date: {date_str}, Weekday: {is_weekday}")
    else:
        print(f"  '{sample}' -> None")

print("\nTesting extract_date_and_weekday_case_null_day (for October):")
for sample in test_samples[7:]:
    result = extract_date_and_weekday_case_null_day(sample)
    if result is not None:
        date_str, is_weekday = result  
        print(f"  '{sample}' -> Date: {date_str}, Weekday: {is_weekday}")
    else:
        print(f"  '{sample}' -> None")

# Preparation OD

In [ ]:
for i,df_od in enumerate([df_od_july,df_od_august,df_od_october]):
    print("################################################################################")
    print("OD: ", list_files_od[i])
    if i == 2:
        is_null_day = True
    else:
        is_null_day = False
    print("Add columns for day and weekday/weekend")
    df_od = add_column_is_week_and_str_day(df_od = df_od,
                                    str_period_id_presenze = str_period_id_presenze,
                                    col_str_day_od = col_str_day_od,
                                    col_str_is_week = col_str_is_week,
                                    is_null_day = is_null_day)
    Tij_dist = df_od.filter(pl.col(str_trip_type_od) != "out-out",
                            pl.col(str_trip_type_od) != "in-out")
                                                                        # filter the dataframe to remove the out-out and in-in trips
    if col_str_day_od not in Tij_dist.columns or col_str_is_week not in Tij_dist.columns:
        print("Add columns for day and weekday/weekend")
        Tij_dist = add_column_is_week_and_str_day(df_od = Tij_dist,
                                                str_period_id_presenze = str_period_id_presenze,
                                                col_str_day_od = col_str_day_od,
                                                col_str_is_week = col_str_is_week,
                                                is_null_day = is_null_day)                                                                                                               # NOTE: download the file from the bucket


    for user_profile in UserProfiles:
        list_unique_days_od = Tij_dist[col_str_day_od].unique().to_list()
        for str_day in list_unique_days_od:
            for str_t,str_t1 in list_time_intervals:
                for is_weekday in ["Feriale","Prefestivo","Festivo"]:
                    # NOTE: Si Rompe qui: Non Filtra sul weekday #
                    print(f"size of OD before weekday filtering: {Tij_dist.shape}")
                    # NOTE: Filter by day and weekday/weekend
                    Tij_dist_week = Tij_dist.filter(pl.col(col_str_is_week) == is_weekday)  # filter the dataframe to keep only the rows with the correct weekday/weekend
                    print("Resulting OD after filtering by weekday/weekend: ", Tij_dist.shape)                                                                                                   # print the shape of the resulting dataframe
                    int_time_hour_start_window_interest = int(str_t)
                    ################### convert_vodafone_format_OD_to_pipeline_format ##########################################################################
                    print(f"Analyzing OD for day {str_day}, time interval {str_t}-{str_t1}, user profile {user_profile}, is_weekday {is_weekday}")
                    Tij_dist_final = convert_vodafone_format_OD_to_pipeline_format(Tij_dist,
                                                                        str_day=str_day,
                                                                        col_str_day=col_str_day_od,
                                                                        int_time_hour_start_window_interest=int_time_hour_start_window_interest,
                                                                        str_t=str_t,
                                                                        str_t1=str_t1,
                                                                        IndexVodafone2UserProfile=UserProfile2IndexVodafone,
                                                                        user_profile_col=str_destination_visitor_class_id_od,
                                                                        user_profile=user_profile,
                                                                        str_trips_od=str_trips_od,
                                                                        str_col_O_vodafone=str_origin_od,
                                                                        str_col_D_vodafone=str_destination_od,
                                                                        str_col_n_trips=str_col_n_trips,
                                                                        str_departure_hour_od = str_departure_hour_od,                                  # NOTE: column with the hour of departure in df_od
                                                                        col_str_day_od = col_str_day_od,                                                # NOTE: column with the day of the trip in df_od
                                                                        str_trip_type_od = str_trip_type_od,                                            # NOTE: column with the type of trip in df
                                                                        col_str_is_week = col_str_is_week,                                              # NOTE: column with the information if the trip is in the weekend
                                                                        str_origin_visitor_class_id_od = str_origin_visitor_class_id_od,                # NOTE: column with the origin class of visitor in df_od
                                                                        str_destination_visitor_class_id_od = str_destination_visitor_class_id_od,      # NOTE: column with the destination class of visitor in df_od
                                                                        str_nationality_class_id_od = str_nationality_class_id_od,                      # NOTE: column with the nationality class of visitor in df_od

                                                                        )
                    print(Tij_dist_final.head())
                    print(Tij_dist_final[col_str_is_week].head())
                    ################### convert_vodafone_format_OD_to_pipeline_format ##########################################################################


In [ ]:
Tij_dist_week = Tij_dist.filter(pl.col(col_str_is_week) == is_weekday)  # filter the dataframe to keep only the rows with the correct weekday/weekend
Tij_dist_week

In [ ]:
Tij_dist_week[col_str_is_week].unique()